<a href="https://colab.research.google.com/github/ajitnova/stocks_analysis/blob/master/trading_fno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade kiteconnect

In [69]:
import logging
from kiteconnect import KiteConnect

logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key="00j0ceqzr4aslm39")



In [2]:
kite


In [71]:
request_token = '3H9KcNcRNT8FlXJGmXOBf3vheQEy7zN3'
api_secret = 'qadl2j9i31ukdom2by63eevxhfg6wcma'

data = kite.generate_session(request_token, api_secret)
kite.set_access_token(data["access_token"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /session/token HTTP/1.1" 200 None


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/1.1" 200 837711


In [8]:
kite.quote('NFO:NIFTY2281115550CE')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NSE%3ANIFTY%2B50 HTTP/1.1" 200 30


{}

In [10]:
kite.ohlc('NSE:NIFTY+50')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ohlc?i=NSE%3ANIFTY%2B50 HTTP/1.1" 200 30


{}

In [12]:
kite.ltp('NSE:NIFTY 50')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None


{'NSE:NIFTY 50': {'instrument_token': 256265, 'last_price': 15972.35}}

In [14]:
type(options)


list

In [72]:
from datetime import date

options = kite.instruments('NFO')
latest_expiry = [options[i] for i in range(len(options)) if options[i]['expiry'] == date(2022, 7, 14)]
nifty_Spot = kite.ltp('NSE:NIFTY 50')['NSE:NIFTY 50']['last_price']
itm_ce = [latest_expiry[i] for i in range(len(latest_expiry)) if latest_expiry[i]['strike'] - nifty_Spot < 0 and latest_expiry[i]['name'] == 'NIFTY' and latest_expiry[i]['instrument_type'] == 'CE']
itm_pe = [latest_expiry[i] for i in range(len(latest_expiry)) if latest_expiry[i]['strike'] - nifty_Spot > 0 and latest_expiry[i]['name'] == 'NIFTY' and latest_expiry[i]['instrument_type'] == 'PE']

itm_ce_vol = 0
itm_pe_vol = 0

trading_ce_quote = ''
trading_pe_quote = ''

for ce in itm_ce:
  symbol = 'NFO:' + ce['tradingsymbol']
  quote = kite.quote(symbol)
  if quote[symbol]['volume'] > itm_ce_vol:
    itm_ce_vol = quote[symbol]['volume']
    trading_ce_quote = ce['tradingsymbol']

for pe in itm_pe:
  symbol = 'NFO:' + pe['tradingsymbol']
  quote = kite.quote(symbol)
  if quote[symbol]['volume'] > itm_pe_vol:
    itm_pe_vol = quote[symbol]['volume']
    trading_pe_quote = pe['tradingsymbol']



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/1.1" 200 837711
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2271413800CE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2271413850CE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2271413900CE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting n

In [73]:
pe = kite.quote('NFO:{}'.format(trading_pe_quote))
ce = kite.quote('NFO:{}'.format(trading_ce_quote))



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2271416000PE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2271415950CE HTTP/1.1" 200 None


In [74]:
if pe['NFO:{}'.format(trading_pe_quote)]['last_price'] > pe['NFO:{}'.format(trading_pe_quote)]['average_price']:
  print("BUYING PE {}".format(trading_pe_quote))

if ce['NFO:{}'.format(trading_ce_quote)]['last_price'] > ce['NFO:{}'.format(trading_ce_quote)]['average_price']:
  print("BUYING CE {}".format(trading_ce_quote))


BUYING PE NIFTY2271416000PE


In [120]:
from datetime import date

def nifty_option_decision():
  options = kite.instruments('NFO')

  latest_expiry = [options[i] for i in range(len(options)) if options[i]['expiry'] == date(2022, 7, 21)]
  nifty_Spot = kite.ltp('NSE:NIFTY 50')['NSE:NIFTY 50']['last_price']

  itm_ce = [latest_expiry[i] for i in range(len(latest_expiry)) if latest_expiry[i]['strike'] - nifty_Spot < 0 and latest_expiry[i]['name'] == 'NIFTY' and latest_expiry[i]['instrument_type'] == 'CE']
  itm_pe = [latest_expiry[i] for i in range(len(latest_expiry)) if latest_expiry[i]['strike'] - nifty_Spot > 0 and latest_expiry[i]['name'] == 'NIFTY' and latest_expiry[i]['instrument_type'] == 'PE']

  itm_ce_vol = 0
  itm_pe_vol = 0

  trading_ce_quote = ''
  trading_pe_quote = ''

  for ce in itm_ce:
    symbol = 'NFO:' + ce['tradingsymbol']
    quote = kite.quote(symbol)
    if quote[symbol]['volume'] > itm_ce_vol:
      itm_ce_vol = quote[symbol]['volume']
      trading_ce_quote = ce['tradingsymbol']

  for pe in itm_pe:
    symbol = 'NFO:' + pe['tradingsymbol']
    quote = kite.quote(symbol)
    if quote[symbol]['volume'] > itm_pe_vol:
      itm_pe_vol = quote[symbol]['volume']
      trading_pe_quote = pe['tradingsymbol']

  pe = kite.quote('NFO:{}'.format(trading_pe_quote))
  ce = kite.quote('NFO:{}'.format(trading_ce_quote))

  if pe['NFO:{}'.format(trading_pe_quote)]['last_price'] > pe['NFO:{}'.format(trading_pe_quote)]['average_price']:
    return {"PE": trading_pe_quote}

  if ce['NFO:{}'.format(trading_ce_quote)]['last_price'] > ce['NFO:{}'.format(trading_ce_quote)]['average_price']:
    return {"CE": trading_ce_quote}



In [122]:
import time as st
from datetime import datetime, time


def nifty_spot_price():
  sleep_time = 1
  spot_price = []
  for i in range(3):
    spot = kite.ltp('NSE:NIFTY 50')
    spot_price.append(spot['NSE:NIFTY 50']['last_price'])
    st.sleep(sleep_time)
  return spot_price

def nifty_spot_decision(spot_price):
  spot_price_change = []
  for idx, price in enumerate(spot_price[1:]):
    if idx == 0:
      continue
    else:
      diff = price - spot_price[idx - 1]
      spot_price_change.append(diff)

  if sum(spot_price_change) > 0 and spot_price[-1] > sum(spot_price)/len(spot_price):
    return "CE"
  if sum(spot_price_change) < 0 and spot_price[-1] < sum(spot_price)/len(spot_price):
    return "PE"


In [134]:
import time
from multiprocessing import Process

def trade():
  now = datetime.now().time()

  open_time_str = '09::15::00'
  close_time_str = '15::30::00'

  open_time = datetime.strptime(open_time_str, '%H::%M::%S').time()
  close_time = datetime.strptime(close_time_str, '%H::%M::%S').time()

  while now < close_time and now > open_time:

    spot_decision = Process(target=nifty_spot_decision(nifty_spot_price()))
    spot_decision.start()
    option_decision = Process(target=nifty_option_decision)
    option_decision.start()
    spot_decision.join()
    option_decision.join()
    print(spot_decision)
    print(option_decision)
    time.sleep()
    # spot_price = nifty_spot_price()
    # spot_decision = nifty_spot_decision(spot_price)
    # option_decision = nifty_option_decision()
    # print(spot_decision)
    # print(option_decision)
    # if spot_decision == list(option_decision.keys())[0]:
    #   kite.place_order(variety='regular', exchange='NFO', tradingsymbol=list(option_decision.values())[0], transaction_type='BUY', quantity=50, product='NRML', order_type='MARKET')

In [135]:
trade()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/1.1" 200 837711
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connec

<Process(Process-1, stopped)>
<Process(Process-2, stopped)>


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/1.1" 200 837711
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:44

<Process(Process-3, stopped)>
<Process(Process-4, stopped)>


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/1.1" 200 837711
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:44

...

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/1.1" 200 837711
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:44

<Process(Process-7, stopped)>
<Process(Process-8, stopped)>


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/1.1" 200 837711
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:44

KeyboardInterrupt: ignored

Process Process-10:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-120-57ee4450f990>", line 20, in nifty_option_decision
    quote = kite.quote(symbol)
  File "/usr/local/lib/python3.7/dist-packages/kiteconnect/connect.py", line 577, in quote
    data = self._get("market.quote", params={"i": ins})
  File "/usr/local/lib/python3.7/dist-packages/kiteconnect/connect.py", line 842, in _get
    return self._request(route, "GET", url_args=url_args, params=params, is_json=is_json)
  File "/usr/local/lib/python3.7/dist-packages/kiteconnect/connect.py", line 894, in _request
    proxies=self.proxies)
  File "/usr/local/lib/python3.7/dist-packages/requests/api.py", line 61, in request
    return session.request(method=method, url=url, **kwargs)
  File "/usr/local/lib/python3.

In [125]:
option_decision = nifty_option_decision()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/1.1" 200 837711
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2272113800CE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2272113850CE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2272113900CE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting n

In [86]:
from multiprocessing import Process

if __name__ == '__main__':
  spot_price_cal = Process(target=nifty_spot_price())
  spot_price_cal.start()
  trade = Process(target=trade)
  trade.start()
  spot_price_cal.join()
  trade.join()

datetime.time

In [131]:
list(option_decision.values())[0]

'NIFTY2272116000PE'

In [109]:
nifty_spot_price

[15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65]

In [110]:
nifty_spot_price

[15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65,
 15938.65]